# Fine-Tuning BERT Hybrid Models
# Hyperparameter Optimization for Best Performers

**Models to Fine-Tune:**
1. **bert_hybrid_fixed** - HR@10: 0.0690, NDCG@10: 0.1447
2. **bert_hybrid_discrete** - HR@10: 0.0655, NDCG@10: 0.1414

**Strategy:**
- Grid search over key hyperparameters
- Track all metrics (HR@10, NDCG@10, MRR)
- Save best configurations
- Quick experiments (30-50 epochs with early stopping)

**Time Estimate:** ~4-6 hours with GPU T4

## Step 1: Setup and Install Dependencies

In [1]:
# Check if we're in Kaggle or Colab
import os
import sys

# If not already in project directory, clone it
if not os.path.exists('length-adaptive'):
    print("📦 Cloning repository...")
    !git clone https://github.com/faroukq1/length-adaptive.git
    %cd length-adaptive
else:
    print("✅ Repository already exists")
    if os.path.basename(os.getcwd()) != 'length-adaptive':
        %cd length-adaptive

print(f"📂 Current directory: {os.getcwd()}")

📦 Cloning repository...
Cloning into 'length-adaptive'...
remote: Enumerating objects: 442, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 442 (delta 123), reused 204 (delta 54), pack-reused 158 (from 1)
Receiving objects: 100% (442/442), 153.25 MiB | 22.48 MiB/s, done.
Resolving deltas: 100% (167/167), done.
/kaggle/working/length-adaptive
📂 Current directory: /kaggle/working/length-adaptive


In [2]:
# Install dependencies
print("📥 Installing dependencies...")
!pip install -q torch-geometric tqdm scikit-learn pandas matplotlib seaborn

print("✅ Dependencies installed!")

📥 Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
✅ Dependencies installed!


## Step 2: Verify GPU and Setup

In [3]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pathlib import Path
import pickle
import json
from tqdm import tqdm

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("="*70)
print(f"🚀 Device: {device}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print("="*70)

🚀 Device: cuda
📊 GPU: Tesla P100-PCIE-16GB
💾 Memory: 15.89 GB


## Step 3: Load Data and Graph

In [4]:
# Add project to path
sys.path.insert(0, os.getcwd())

from src.data.dataloader import get_dataloaders

print("📂 Loading MovieLens-1M data...")

# Load data - get_dataloaders returns all three loaders plus config
train_loader, val_loader, test_loader, config = get_dataloaders(
    data_path='data/ml-1m/processed/sequences.pkl',
    batch_size=256,
    max_len=200,
    num_workers=2  # Reduce workers for Kaggle/Colab
)

# Get number of items from config
num_items = config['num_items']

print(f"✅ Data loaded successfully!")
print(f"📊 Number of items: {num_items}")
print(f"📊 Train batches: {len(train_loader)}")
print(f"📊 Validation batches: {len(val_loader)}")
print(f"📊 Test batches: {len(test_loader)}")

# Load graph for GNN component
print("\n📊 Loading co-occurrence graph...")
with open('data/graphs/cooccurrence_graph.pkl', 'rb') as f:
    graph_data = pickle.load(f)

edge_index = graph_data['edge_index']
edge_weight = graph_data['edge_weight']
print(f"✅ Graph loaded: {edge_index.shape[1]:,} edges")

📂 Loading MovieLens-1M data...
✅ Data loaded successfully!
📊 Number of items: 3533
📊 Train batches: 2177
📊 Validation batches: 24
📊 Test batches: 24

📊 Loading co-occurrence graph...
✅ Graph loaded: 151,874 edges


## Step 4: Define Fine-Tuning Configuration

**Hyperparameters to Tune:**
1. Learning rate: [0.0005, 0.001, 0.002]
2. d_model (embedding size): [64, 128]
3. n_heads: [2, 4]
4. n_blocks: [2, 3]
5. Dropout: [0.1, 0.2]
6. For discrete: L_short, L_long values
7. For fixed: alpha values

In [5]:
# Hyperparameter search space
hyperparameter_configs = {
    'bert_hybrid_fixed': [
        # Original config
        # {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.2, 'alpha': 0.5},
        # Vary alpha
        # {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.2, 'alpha': 0.3},
        # {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.2, 'alpha': 0.7},
        # Vary learning rate
        # {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.0005, 'dropout': 0.2, 'alpha': 0.5},
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.002, 'dropout': 0.2, 'alpha': 0.5},
        # Larger model
        {'d_model': 128, 'n_heads': 4, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.2, 'alpha': 0.5},
        # Deeper model
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 3, 'lr': 0.001, 'dropout': 0.2, 'alpha': 0.5},
        # Lower dropout
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.1, 'alpha': 0.5},
    ],
    
    'bert_hybrid_discrete': [
        # Original config
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.2, 'L_short': 10, 'L_long': 30},
        # Vary thresholds
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.2, 'L_short': 5, 'L_long': 20},
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.2, 'L_short': 15, 'L_long': 40},
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.2, 'L_short': 8, 'L_long': 25},
        # Vary learning rate
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.0005, 'dropout': 0.2, 'L_short': 10, 'L_long': 30},
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.002, 'dropout': 0.2, 'L_short': 10, 'L_long': 30},
        # Larger model
        {'d_model': 128, 'n_heads': 4, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.2, 'L_short': 10, 'L_long': 30},
        # Deeper model
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 3, 'lr': 0.001, 'dropout': 0.2, 'L_short': 10, 'L_long': 30},
        # Lower dropout
        {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.001, 'dropout': 0.1, 'L_short': 10, 'L_long': 30},
    ]
}

print("="*70)
print("🔬 Hyperparameter Search Space")
print("="*70)
print(f"\nbert_hybrid_fixed: {len(hyperparameter_configs['bert_hybrid_fixed'])} configurations")
print(f"bert_hybrid_discrete: {len(hyperparameter_configs['bert_hybrid_discrete'])} configurations")
print(f"\nTotal experiments: {sum(len(v) for v in hyperparameter_configs.values())}")

🔬 Hyperparameter Search Space

bert_hybrid_fixed: 4 configurations
bert_hybrid_discrete: 9 configurations

Total experiments: 13


## Step 5: Training Functions

In [6]:
from src.models.bert4rec_hybrid import HybridBERT4RecGNN
from src.train.trainer import Trainer
from src.train.loss import BPRLoss

def create_model(model_type, num_items, config):
    """Create model with given configuration"""
    fusion_type = model_type.replace('bert_hybrid_', '')
    
    # Base parameters
    model_params = {
        'num_items': num_items,
        'd_model': config['d_model'],
        'n_heads': config['n_heads'],
        'n_blocks': config['n_blocks'],
        'd_ff': config['d_model'] * 4,  # Standard transformer ratio
        'max_len': 200,
        'gnn_layers': 2,
        'dropout': config['dropout'],
        'fusion_type': fusion_type,
    }
    
    # Add fusion-specific parameters
    if fusion_type == 'fixed':
        model_params['fixed_alpha'] = config['alpha']
    elif fusion_type == 'discrete':
        model_params['L_short'] = config['L_short']
        model_params['L_long'] = config['L_long']
    
    return HybridBERT4RecGNN(**model_params)

def train_model(model_type, config, epochs=50, patience=10):
    """Train a single model configuration"""
    
    print("\n" + "="*70)
    print(f"🚀 Training: {model_type}")
    print("="*70)
    print(f"Config: {config}")
    
    # Create model
    model = create_model(model_type, num_items, config).to(device)
    
    # Setup training
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        edge_index=edge_index,
        edge_weight=edge_weight,
        device=device,
        lr=config['lr'],
        patience=patience,
        save_dir='results/finetuning/checkpoints'
    )
    
    # Train
    history = trainer.train(num_epochs=epochs, eval_every=5)
    
    # Get best results
    best_epoch = history['best_epoch']
    best_metrics = history['val_metrics'][best_epoch]
    
    print(f"\n✅ Training Complete!")
    print(f"📊 Best Epoch: {best_epoch}")
    print(f"📈 HR@10: {best_metrics['hr@10']:.6f}")
    print(f"📈 NDCG@10: {best_metrics['ndcg@10']:.6f}")
    print(f"📈 MRR: {best_metrics['mrr']:.6f}")
    
    return {
        'model_type': model_type,
        'config': config,
        'history': history,
        'best_metrics': best_metrics,
        'best_epoch': best_epoch,
        'model': model
    }

print("✅ Training functions defined!")

✅ Training functions defined!


## Step 6: Run Fine-Tuning Experiments

This will train all configurations. Each run takes ~15-25 minutes with early stopping.

In [ ]:
# Store all results
all_results = []

# Create results directory
results_dir = Path('results/finetuning')
results_dir.mkdir(parents=True, exist_ok=True)

print("="*70)
print("🔬 Starting Fine-Tuning Experiments")
print("="*70)
print(f"📁 Results will be saved to: {results_dir}")

# Fine-tune bert_hybrid_fixed
print("\n" + "="*70)
print("🎯 BERT Hybrid Fixed Models")
print("="*70)

for i, config in enumerate(hyperparameter_configs['bert_hybrid_fixed'], 1):
    print(f"\n📍 Configuration {i}/{len(hyperparameter_configs['bert_hybrid_fixed'])}")
    
    try:
        result = train_model('bert_hybrid_fixed', config, epochs=50, patience=10)
        all_results.append(result)
        
        # Save intermediate results
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        result_file = results_dir / f'bert_hybrid_fixed_config{i}_{timestamp}.pkl'
        with open(result_file, 'wb') as f:
            pickle.dump(result, f)
        
        print(f"💾 Saved to: {result_file}")
        
    except Exception as e:
        print(f"❌ Error in configuration {i}: {e}")
        continue

# Fine-tune bert_hybrid_discrete
print("\n" + "="*70)
print("🎯 BERT Hybrid Discrete Models")
print("="*70)

for i, config in enumerate(hyperparameter_configs['bert_hybrid_discrete'], 1):
    print(f"\n📍 Configuration {i}/{len(hyperparameter_configs['bert_hybrid_discrete'])}")
    
    try:
        result = train_model('bert_hybrid_discrete', config, epochs=50, patience=10)
        all_results.append(result)
        
        # Save intermediate results
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        result_file = results_dir / f'bert_hybrid_discrete_config{i}_{timestamp}.pkl'
        with open(result_file, 'wb') as f:
            pickle.dump(result, f)
        
        print(f"💾 Saved to: {result_file}")
        
    except Exception as e:
        print(f"❌ Error in configuration {i}: {e}")
        continue

print("\n" + "="*70)
print("✅ All Fine-Tuning Experiments Complete!")
print("="*70)
print(f"📊 Total successful runs: {len(all_results)}")

🔬 Starting Fine-Tuning Experiments
📁 Results will be saved to: results/finetuning

🎯 BERT Hybrid Fixed Models

📍 Configuration 1/4

🚀 Training: bert_hybrid_fixed
Config: {'d_model': 64, 'n_heads': 2, 'n_blocks': 2, 'lr': 0.002, 'dropout': 0.2, 'alpha': 0.5}
STARTING TRAINING
Device: cuda
Model parameters: 569,408
Training batches: 2177
Validation batches: 24



Epoch 5: 100%|██████████| 2177/2177 [01:49<00:00, 19.97it/s, loss=0.1925]



[Epoch 5] Evaluating...


Evaluating: 100%|██████████| 24/24 [00:00<00:00, 30.18it/s]



[Epoch 5/50] Time: 109.9s
  Train Loss: 0.2541
  Val HR@10: 0.0262
  Val NDCG@10: 0.0113
  Val MRR@10: 0.0070
  ✓ New best! (0.0000 → 0.0113)


...

Epoch 10: 100%|██████████| 2177/2177 [01:50<00:00, 19.75it/s, loss=0.1182]



[Epoch 10] Evaluating...


Evaluating: 100%|██████████| 24/24 [00:00<00:00, 31.27it/s]



[Epoch 10/50] Time: 111.0s
  Train Loss: 0.1180
  Val HR@10: 0.0902
  Val NDCG@10: 0.0426
  Val MRR@10: 0.0285
  ✓ New best! (0.0404 → 0.0426)


Epoch 12:  93%|█████████▎| 2023/2177 [01:42<00:07, 19.83it/s, loss=0.0970]

## Step 7: Analyze Results

In [ ]:
# Create results DataFrame
results_data = []

for result in all_results:
    row = {
        'Model': result['model_type'],
        'HR@10': result['best_metrics']['hr@10'],
        'NDCG@10': result['best_metrics']['ndcg@10'],
        'MRR': result['best_metrics']['mrr'],
        'Best Epoch': result['best_epoch'],
        **result['config']  # Add all config parameters
    }
    results_data.append(row)

results_df = pd.DataFrame(results_data)

# Sort by NDCG@10
results_df = results_df.sort_values('NDCG@10', ascending=False)

print("="*70)
print("📊 Fine-Tuning Results Summary")
print("="*70)
print(results_df.to_string(index=False))

# Save results
results_csv = results_dir / 'finetuning_results.csv'
results_df.to_csv(results_csv, index=False)
print(f"\n💾 Results saved to: {results_csv}")

In [ ]:
# Identify best configurations
print("="*70)
print("🏆 Best Configurations")
print("="*70)

for model_type in ['bert_hybrid_fixed', 'bert_hybrid_discrete']:
    model_results = results_df[results_df['Model'] == model_type]
    if len(model_results) > 0:
        best_idx = model_results['NDCG@10'].idxmax()
        best = model_results.loc[best_idx]
        
        print(f"\n{model_type}:")
        print(f"  HR@10: {best['HR@10']:.6f}")
        print(f"  NDCG@10: {best['NDCG@10']:.6f}")
        print(f"  MRR: {best['MRR']:.6f}")
        print(f"  Configuration:")
        for key, value in best.items():
            if key not in ['Model', 'HR@10', 'NDCG@10', 'MRR', 'Best Epoch']:
                print(f"    {key}: {value}")

## Step 8: Visualizations

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. NDCG@10 comparison
ax = axes[0, 0]
for model_type in ['bert_hybrid_fixed', 'bert_hybrid_discrete']:
    model_data = results_df[results_df['Model'] == model_type]
    ax.scatter(range(len(model_data)), model_data['NDCG@10'], 
              label=model_type, s=100, alpha=0.7)
ax.set_xlabel('Configuration Index', fontsize=12)
ax.set_ylabel('NDCG@10', fontsize=12)
ax.set_title('NDCG@10 across Configurations', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. HR@10 vs NDCG@10
ax = axes[0, 1]
for model_type in ['bert_hybrid_fixed', 'bert_hybrid_discrete']:
    model_data = results_df[results_df['Model'] == model_type]
    ax.scatter(model_data['HR@10'], model_data['NDCG@10'], 
              label=model_type, s=100, alpha=0.7)
ax.set_xlabel('HR@10', fontsize=12)
ax.set_ylabel('NDCG@10', fontsize=12)
ax.set_title('HR@10 vs NDCG@10', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Learning rate effect (for fixed)
ax = axes[1, 0]
if 'lr' in results_df.columns:
    fixed_data = results_df[results_df['Model'] == 'bert_hybrid_fixed']
    if len(fixed_data) > 0:
        lr_groups = fixed_data.groupby('lr')['NDCG@10'].mean()
        ax.bar(range(len(lr_groups)), lr_groups.values)
        ax.set_xticks(range(len(lr_groups)))
        ax.set_xticklabels([f'{lr:.4f}' for lr in lr_groups.index])
        ax.set_xlabel('Learning Rate', fontsize=12)
        ax.set_ylabel('Average NDCG@10', fontsize=12)
        ax.set_title('Learning Rate Effect (Fixed)', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')

# 4. Model size effect
ax = axes[1, 1]
if 'd_model' in results_df.columns:
    size_groups = results_df.groupby('d_model')['NDCG@10'].mean()
    ax.bar(range(len(size_groups)), size_groups.values, color='coral')
    ax.set_xticks(range(len(size_groups)))
    ax.set_xticklabels([f'{size}' for size in size_groups.index])
    ax.set_xlabel('Model Size (d_model)', fontsize=12)
    ax.set_ylabel('Average NDCG@10', fontsize=12)
    ax.set_title('Model Size Effect', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(results_dir / 'finetuning_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualizations created and saved!")

## Step 9: Test Best Models

Evaluate the best configuration of each model on the test set.

In [ ]:
from src.eval.metrics import evaluate_model

# Evaluate best models on test set
print("="*70)
print("🎯 Testing Best Configurations")
print("="*70)

test_results = []

for model_type in ['bert_hybrid_fixed', 'bert_hybrid_discrete']:
    model_results = results_df[results_df['Model'] == model_type]
    if len(model_results) == 0:
        continue
        
    # Get best configuration
    best_idx = model_results['NDCG@10'].idxmax()
    best_result = [r for r in all_results if r['model_type'] == model_type][best_idx]
    
    print(f"\n{model_type}:")
    print(f"  Best config: {best_result['config']}")
    
    # Evaluate on test set
    model = best_result['model'].to(device)
    model.eval()
    
    test_metrics = evaluate_model(model, test_loader, device, k=10)
    
    print(f"\n  Test Results:")
    print(f"    HR@10: {test_metrics['hr@10']:.6f}")
    print(f"    NDCG@10: {test_metrics['ndcg@10']:.6f}")
    print(f"    MRR: {test_metrics['mrr']:.6f}")
    
    test_results.append({
        'Model': model_type,
        'Test_HR@10': test_metrics['hr@10'],
        'Test_NDCG@10': test_metrics['ndcg@10'],
        'Test_MRR': test_metrics['mrr'],
        'Config': str(best_result['config'])
    })

# Save test results
test_df = pd.DataFrame(test_results)
test_csv = results_dir / 'test_results_best_configs.csv'
test_df.to_csv(test_csv, index=False)

print("\n" + "="*70)
print("✅ Test evaluation complete!")
print("="*70)
print(test_df.to_string(index=False))

## Step 10: Save Best Models

In [ ]:
# Save best models
models_dir = results_dir / 'best_models'
models_dir.mkdir(exist_ok=True)

print("="*70)
print("💾 Saving Best Models")
print("="*70)

for model_type in ['bert_hybrid_fixed', 'bert_hybrid_discrete']:
    model_results = results_df[results_df['Model'] == model_type]
    if len(model_results) == 0:
        continue
        
    # Get best result
    best_idx = model_results['NDCG@10'].idxmax()
    best_result = [r for r in all_results if r['model_type'] == model_type][best_idx]
    
    # Save model
    model_path = models_dir / f'{model_type}_best.pt'
    torch.save({
        'model_state_dict': best_result['model'].state_dict(),
        'config': best_result['config'],
        'metrics': best_result['best_metrics'],
        'epoch': best_result['best_epoch']
    }, model_path)
    
    print(f"✅ Saved {model_type} to {model_path}")
    print(f"   NDCG@10: {best_result['best_metrics']['ndcg@10']:.6f}")

print("\n💾 All models saved successfully!")

## Step 11: Download Results

Package all results for download.

In [ ]:
import shutil

# Create zip file
print("📦 Creating results package...")

zip_path = '/tmp/finetuning_results'
shutil.make_archive(zip_path, 'zip', results_dir)

print(f"✅ Results packaged: {zip_path}.zip")
print("\n📥 Download the file to get all results!")
print(f"\nIncluded:")
print(f"  - Individual experiment results (pkl files)")
print(f"  - Summary CSV: finetuning_results.csv")
print(f"  - Test results: test_results_best_configs.csv")
print(f"  - Visualizations: finetuning_analysis.png")
print(f"  - Best models: best_models/*.pt")

## Summary

**Fine-Tuning Complete! 🎉**

This notebook has:
1. ✅ Trained multiple configurations of bert_hybrid_fixed and bert_hybrid_discrete
2. ✅ Tested variations in: learning rate, model size, depth, dropout, fusion parameters
3. ✅ Identified best configurations for each model
4. ✅ Evaluated best models on test set
5. ✅ Saved all results and best model checkpoints

**Next Steps:**
- Review the results CSV to see all configurations
- Check the visualizations for insights
- Use the best models for production or further experiments
- Consider testing on other datasets